In [2]:
print("bonjour")

bonjour


In [3]:
!pip install numpy tensorflow scikit-learn seaborn matplotlib

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b7/98/5640a09daa3abf0caeaefa6e7bf0d10c0aa28a77c84e507d6a716e0e23df/numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/59.7 kB 640.0 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/59.7 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 59.7/59.7 kB 626.6 kB/s eta 0:00:00
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/48/a7/2f03f6de3c4976db6d2a898c408feb491ca10399af1f6039d6bef3e6ba6a/tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/aa/ce/c0b912f2f31aeb1b756a6ba56bcd84dd1f8a148470526a48515a3f4d48cd/scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for seaborn from


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ------ --------------------------------- 64.2/385.2 MB 1.6 MB/s eta 0:03:16
   ------ --------------------------------- 64.3/385.2 MB 1.6 MB/s eta 0:03:16
   ------ --------------------------------- 64.3/385.2 MB 1.6 MB/s eta 0:03:17
   ------ --------------------------------- 64.3/385.2 MB 1.6 MB/s eta 0:03:17
   ------ --------------------------------- 64.4/385.2 MB 1.6 MB/s eta 0:03:18
   ------ --------------------------------- 64.4/385.2 MB 1.6 MB/s eta 0:03:19
   ------ --------------------------------- 64.5/385.2 MB 1.6 MB/s eta 0:03:19
   ------ --------------------------------- 64.5/385.2 MB 1.6 MB/s eta 0:03:19
   ------ --------------------------------- 64.6/385.2 MB 1.6 MB/s eta 0:03:20
   ------ --------------------------------- 64.7/385.2 MB 1.6 MB/s eta 0:03:20
   ------ --------------------------------- 64.7/385.2 MB 1.6 MB/s eta 0:03:20
   ------ --------------------------------- 64.8/385.2 MB 1.6 MB/s eta 0:03:21
   ------ --------------------------------- 64.9/38

In [8]:

import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [9]:
# Paths
IMG_SAVE_PATH = 'data/train'
IMG_SAVE_PATH_TESTING = 'data/val'
# Mapping from string to integer labels
Str_to_Int = {
    'Actinic keratosis': 0,
    'Basal cell carcinoma': 1,
    'Benign keratosis': 2,
    'Dermatofibroma': 3,
    'Melanocytic nevus': 4,
    'Melanoma': 5,
    'Squamous cell carcinoma': 6,
    'Vascular lesion': 7
}
NUM_CLASSES = len(Str_to_Int)

# Function to map string labels to integers
def str_to_Int_mapper(val):
    return Str_to_Int[val]

In [10]:
# Data Preparation
def load_data(path):
    dataset = []
    for directory in os.listdir(path):
        directory_path = os.path.join(path, directory)
        for image_file in os.listdir(directory_path):
            image_path = os.path.join(directory_path, image_file)
            try:
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(240, 240))
                img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
                dataset.append([img_array, directory])
            except FileNotFoundError:
                print(f'Image file not found: {image_path}. Skipping...')
    return zip(*dataset)

In [11]:
# Load training data
data, labels = load_data(IMG_SAVE_PATH)
labels = np.array([str_to_Int_mapper(label) for label in labels])
data = np.array(data)
labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

In [12]:
# Load testing data
testing_data, testing_labels = load_data(IMG_SAVE_PATH_TESTING)
testing_labels = np.array([str_to_Int_mapper(label) for label in testing_labels])
testing_data = np.array(testing_data)
testing_labels = tf.keras.utils.to_categorical(testing_labels, num_classes=NUM_CLASSES)

In [13]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow(data, labels, batch_size=32)

In [14]:
# Create Transfer Learning Model
def create_transfer_learning_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(240, 240, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Unfreeze some layers of the base model for fine-tuning
    for layer in base_model.layers:
        layer.trainable = True
    
    return model

model = create_transfer_learning_model()

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


In [15]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
]

In [16]:
# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train Model
history = model.fit(
    train_generator,
    epochs=30,
    steps_per_epoch=len(data) // 32,
    validation_data=(testing_data, testing_labels),
    callbacks=callbacks
)

Epoch 1/30


D:\Users\moham\PycharmProjects\DERMA-TN\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


In [ ]:
# Plot Training History
def plot_acc(history):
    sns.set()
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    sns.lineplot(x=history.epoch, y=history.history['accuracy'], label='train', ax=axes[0])
    sns.lineplot(x=history.epoch, y=history.history['val_accuracy'], label='valid', ax=axes[0])
    axes[0].set_title('Accuracy')
    
    sns.lineplot(x=history.epoch, y=history.history['loss'], label='train', ax=axes[1])
    sns.lineplot(x=history.epoch, y=history.history['val_loss'], label='valid', ax=axes[1])
    axes[1].set_title('Loss')
    
    plt.tight_layout()
    plt.show()

plot_acc(history)

In [ ]:
# Save the Model
model.save("mobile_net.h5")

In [ ]:
# Evaluate Model
test_predictions = model.predict(testing_data)
y_true = np.argmax(testing_labels, axis=1)
y_pred = np.argmax(test_predictions, axis=1)
# Print classification report
print(classification_report(y_true, y_pred, target_names=Str_to_Int.keys()))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=Str_to_Int.keys(), yticklabels=Str_to_Int.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()